# Use DCP as initializer
---
## Define dataset

In [4]:
import numpy as np
from dcp import train, test
from data import SceneNet
from torch.utils.data import DataLoader
import sys
sys.path.append("dcp-master")
from model import DCP
import torch
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import k3d
from util import transform_point_cloud

def visualize_pointcloud(point_cloud1, point_size, point_cloud2=None, flip_axes=False, name='point_cloud', R=None, t=None):
    plot = k3d.plot(name=name, grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
    plt_points1 = k3d.points(positions=point_cloud1, point_size=point_size, color=0xd0d0d0)
    plot += plt_points1
    plt_points1.shader = '3d'
    if point_cloud2 is not None:
        plt_points2 = k3d.points(positions=point_cloud2, point_size=point_size, color=0x0dd00d)
        plot += plt_points2
        plt_points2.shader = '3d'
    plot.display()

def transform(point_cloud, R=None, t=None):
    t_broadcast = np.broadcast_to(t[:, np.newaxis], (3, point_cloud.shape[0]))
    return (R @ point_cloud.T + t_broadcast).T

trainDataset = SceneNet(1024, "train", filter=False)
valDataset = SceneNet(1024, "val", filter=False)
train_loader = DataLoader(trainDataset, batch_size=32, shuffle=False, drop_last=False)
test_loader = DataLoader(valDataset, batch_size=1, shuffle=False, drop_last=False)
print(len(trainDataset))
print(len(valDataset))

299
299
0.0


## Define parameters

In [21]:
args = {
        "model_path": 'checkpoints/dcp_v1/models/model.best.t7',
        # "model_path": 'dcp-master/pretrained/dcp_v2.t7',
        "exp_name":"dcp_v1",
         "model":"dcp", 
         "emb_nn":"dgcnn", 
         "pointer":"identity", 
         "head":"svd", 
         "eval": True,
         'emb_dims': 512,
         'cycle': False,
         'use_sgd': False,
         'lr': 0.001,
         'epochs': 350,
         'n_blocks': 1,
         'dropout': 0.0,
         'ff_dims': 1024,
         'n_heads': 4,
         'use_sgd': False,
         'momentum': 0.9,
        }
net = DCP(args)

## DCP Training

In [3]:

train(args, net, train_loader, test_loader)

Use Adam
Epoch 0: -train- Loss: 2.449583
Epoch 1: -train- Loss: 1.590965
Epoch 2: -train- Loss: 1.429964
Epoch 3: -train- Loss: 1.294368
Epoch 4: -train- Loss: 1.193472
Epoch 5: -train- Loss: 1.087186
Epoch 6: -train- Loss: 0.998024
Epoch 7: -train- Loss: 0.930252
Epoch 8: -train- Loss: 0.869094
Epoch 9: -train- Loss: 0.816797
Epoch 10: -train- Loss: 0.757757
Epoch 11: -train- Loss: 0.750793
Epoch 12: -train- Loss: 0.704089
Epoch 13: -train- Loss: 0.754186
Epoch 14: -train- Loss: 0.695255
Epoch 15: -train- Loss: 0.687837
Epoch 16: -train- Loss: 0.674733
Epoch 17: -train- Loss: 0.569572
Epoch 18: -train- Loss: 0.540720
Epoch 19: -train- Loss: 0.495528
Epoch 20: -train- Loss: 0.481770
Epoch 21: -train- Loss: 0.496051
Epoch 22: -train- Loss: 0.454704
Epoch 23: -train- Loss: 0.471385
Epoch 24: -train- Loss: 0.543799
Epoch 25: -train- Loss: 0.590928
Epoch 26: -train- Loss: 0.584126
Epoch 27: -train- Loss: 0.735792
Epoch 28: -train- Loss: 0.686409
Epoch 29: -train- Loss: 0.635356
Epoch 30: -

## DCP Testing

In [22]:
net.load_state_dict(torch.load(args['model_path']), strict=False)

r, t = test(args, net, test_loader)
print(r[0])
print(t[0])

100%|██████████| 299/299 [00:02<00:00, 138.58it/s]


==FINAL TEST==
A--------->B
EPOCH:: 1, Loss: 0.221726, Cycle Loss: 0.000000, MSE: 2.440407, RMSE: 1.562180, MAE: 1.020274, rot_MSE: 133.800919, rot_RMSE: 11.567235, rot_MAE: 4.192022, trans_MSE: 0.207820, trans_RMSE: 0.455872, trans_MAE: 0.182717
B--------->A
EPOCH:: 1, Loss: 0.221726, MSE: 2.440407, RMSE: 1.562180, MAE: 1.019776, rot_MSE: 133.268372, rot_RMSE: 11.544192, rot_MAE: 4.197148, trans_MSE: 0.202790, trans_RMSE: 0.450323, trans_MAE: 0.183058
[[-0.0740141  -0.8204512   0.5669067 ]
 [-0.96988523  0.19149436  0.15051201]
 [-0.2320468  -0.53869325 -0.80991614]]
[-0.9064398  1.7053207  6.9145575]


## Visualization

In [28]:

valDataset = SceneNet(20000, "val", filter=True)
src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = valDataset[42]
print(rotation_ab)
print(translation_ab)
# print(points1.shape)
transformed_src = transform_point_cloud(torch.tensor(src), torch.tensor(r[42]).unsqueeze(0), torch.tensor(t[42]).unsqueeze(0)).T
transformed_src2 = transform_point_cloud(torch.tensor(src), torch.tensor(rotation_ab).unsqueeze(0), torch.tensor(translation_ab).unsqueeze(0)).T
# transformed_src1 = transform_point_cloud(torch.tensor(src), r1, t1).T
# visualize_pointcloud(target.T, .03, transformed_src1)
visualize_pointcloud(target.T, .03, transformed_src)
visualize_pointcloud(target.T, .03, transformed_src2)
visualize_pointcloud(target.T, .03, src.T)
export_pointcloud_to_obj('71_origin_filter.obj', target.T, np.array(src.T))
# export_pointcloud_to_obj('71_dcp_filter.obj', target.T, np.array(transformed_src))

[[ 0.999362   -0.01426313 -0.03274392]
 [ 0.01678179  0.9968136   0.07798077]
 [ 0.03152734 -0.07848052  0.996417  ]]
[ 0.2539836  -0.06697565  0.05225628]


Output()

Output()

Output()

# ICP Calibration

In [24]:
from icp import test

valDataset = SceneNet(1024, "val", icp=True, r=r, t=t, filter=True)
print(len(valDataset))
r_icp, t_icp = test(valDataset)

299


  0%|          | 0/299 [00:00<?, ?it/s]


SimpleICPException: Too few correspondences! At least 6 correspondences are needed to estimate the 6 rigid body transformation parameters. The current number of correspondences is 0.

In [25]:
# import open3d as o3d
# import numpy as np
# from torch.utils.data import Dataset
# from transformation import transformation
# from scipy.spatial.transform import Rotation
# class TUM(Dataset):
#         def __init__(self, size, r=None, t=None):
#                 self.pcd1 = o3d.io.read_point_cloud("scenenet/val/0/71/1050_after_filtering.pcd")
#                 # print(self.index)

#                 self.pcd2 = o3d.io.read_point_cloud("scenenet/val/0/71/1075_after_filtering.pcd")

#                 index1 = np.random.randint(len(self.pcd1.points), size=size)
#                 index2 = np.random.randint(len(self.pcd2.points), size=size)

#                 self.points1 = np.array(self.pcd1.points)[index1]
#                 self.points2 = np.array(self.pcd2.points)[index2]
                
#                 self.points1 = transform(self.points1, r[42], t[42])
#                 self.points1 = self.points1[np.newaxis,:]
                
    
#         def __getitem__(self, index):
#                 points1 = self.points1[index]
#                 T = transformation(71, 1050, 1075)
#                 R_ab = T[:3,:3]
#                 R_ba = R_ab.T
#                 translation_ab = T[:3,3]
#                 translation_ba = -R_ba.dot(translation_ab)


#                 euler_ab = npmat2euler(R_ab)
#                 euler_ba = npmat2euler(R_ba)
                
#                 return points1.T.astype('float32'), self.points2.T.astype('float32'), R_ab.astype('float32'), \
#         translation_ab.astype('float32'), R_ba.astype('float32'), translation_ba.astype('float32'), \
#         euler_ab.astype('float32'), euler_ba.astype('float32')

#         def __len__(self):
#                 return len(self.points1)
# def npmat2euler(mats, seq='zyx'):
#     eulers = []
#     r = Rotation.from_matrix(mats)
#     eulers.append(r.as_euler(seq, degrees=True))
#     return np.asarray(eulers, dtype='float32')
# data = TUM(15000, r, t)
# r_icp, t_icp = test(data)

100%|██████████| 1/1 [00:02<00:00,  2.32s/it]

==FINAL TEST==
A--------->B
EPOCH:: -1, Loss: 0.019120, Cycle Loss: 0.000000, MSE: 0.108258, RMSE: 0.329026, MAE: 0.244658, rot_MSE: 18.999233, rot_RMSE: 4.358811, rot_MAE: 3.614784, trans_MSE: 0.015213, trans_RMSE: 0.123343, trans_MAE: 0.108211
B--------->A
EPOCH:: -1, Loss: 0.019120, MSE: 0.108258, RMSE: 0.329026, MAE: 0.242663, rot_MSE: 3.591077, rot_RMSE: 1.895014, rot_MAE: 1.783000, trans_MSE: 0.015933, trans_RMSE: 0.126227, trans_MAE: 0.108450


## Final Visualisation

In [26]:
valDataset = SceneNet(20000, "val", icp=True, r=r, t=t, filter=True)
src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = valDataset[42]
print(rotation_ab)
print(translation_ab)
# print(points1.shape)
transformed_src = transform_point_cloud(torch.tensor(src).double(), torch.tensor(r_icp[0]).unsqueeze(0).double(), torch.tensor(t_icp[0]).unsqueeze(0).double()).T
transformed_src2 = transform_point_cloud(torch.tensor(src), torch.tensor(rotation_ab).unsqueeze(0), torch.tensor(translation_ab).unsqueeze(0)).T
# transformed_src1 = transform_point_cloud(torch.tensor(src), r1, t1).T
# visualize_pointcloud(target.T, .03, transformed_src1)
visualize_pointcloud(target.T, .03, transformed_src)
visualize_pointcloud(target.T, .03, transformed_src2)
visualize_pointcloud(target.T, .03, src.T)

export_pointcloud_to_obj('71_icp_filter.obj', target.T, np.array(transformed_src))
# export_pointcloud_to_obj('24_icp.obj', target.T, np.array(transformed_src))

[[ 0.999362   -0.01426313 -0.03274392]
 [ 0.01678179  0.9968136   0.07798077]
 [ 0.03152734 -0.07848052  0.996417  ]]
[ 0.2539836  -0.06697565  0.05225628]


Output()

Output()

Output()

## Export

In [27]:
"""Export to disk"""


def export_mesh_to_obj(path, vertices, faces, vertices2):
    """
    exports mesh as OBJ
    :param path: output path for the OBJ file
    :param vertices: Nx3 vertices
    :param faces: Mx3 faces
    :return: None
    """

    # write vertices starting with "v "
    # write faces starting with "f "

    # ###############
    # DONE: Implement
    v = ""
    f = ""
    v2 = ""
    file = open(path, 'w+')

    if vertices is not None:
        for vertice in vertices:
            v = v + "v "
            for i in vertice:
                v = v + str(i) + " "
            v = v + "0.098039 0.8117647 0.\n"
            # v = v + "\n"
        
    count = 1
    if faces is not None:
        for face in faces:
            f = f + "f "
            for i in face:
                f = f + str(i+1) + " "
            f = f + "\n"

    if vertices2 is not None:
        for vertice2 in vertices2:
            v2 = v2 + "v "
            for i in vertice2:
                v2 = v2 + str(i) + " "
            v2 = v2 + "0.717647 0.717647 0.717647\n"  
            # v2 = v2 + "\n"    
    file.write(v)
    file.write(v2)
    file.write(f)
    file.close()
        
    # ###############


def export_pointcloud_to_obj(path, pointcloud, pointcloud2=None):
    """
    export pointcloud as OBJ
    :param path: output path for the OBJ file
    :param pointcloud: Nx3 points
    :return: None
    """

    # ###############
    # DONE: Implement
    export_mesh_to_obj(path, pointcloud, None, pointcloud2)
    # ###############

# print(rigid_body_transformation_params)
# valDataset = SceneNet(10000, "val", icp=True, r=r, t=t)
# for i in range(len(valDataset)):
#     src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = valDataset[i]
#     transformed_src = transform_point_cloud(torch.tensor(src).double(), torch.tensor(r_icp[i]).unsqueeze(0).double(), torch.tensor(t_icp[i]).unsqueeze(0).double()).T
#     export_pointcloud_to_obj('56_'+str(i)+'.obj', target.T, np.array(transformed_src))